# 03 Azure Cost

In [0]:
from pyspark.sql.functions import col, to_date, concat, lit, from_json
from pyspark.sql.types import MapType, StringType

It is recommended to use [Azure Cost Management Exports](https://learn.microsoft.com/en-us/azure/cost-management-billing/costs/tutorial-improved-exports) to periodically export cost data in CSV format and establish [connections between Unity Catalog and Azure Storage Account](https://learn.microsoft.com/en-us/azure/databricks/connect/unity-catalog/).

In [0]:
csv_path = "/Volumes/main/billing/azure/cost/cost-amortized-cost/*/*/*.csv"
df = spark.read.option("escape", '"').csv(csv_path, header=True)

json_schema = MapType(StringType(), StringType())

df = (
    df.select("Date", "MeterCategory", "CostInBillingCurrency", "Tags", "SubscriptionId")
    .withColumn("Date", to_date(col("Date"), "MM/dd/yyyy"))
    .withColumn("CostInBillingCurrency", col("CostInBillingCurrency").cast("decimal(38,18)"))
    .withColumn("Tags", concat(lit("{"), col("Tags"), lit("}")))
    .withColumn("Tags", from_json(col("Tags"), json_schema))
)

df = df.select(
        col("Date").alias("usage_date"),
        col("MeterCategory").alias("meter_category"),
        (col("CostInBillingCurrency") / 30).alias("cost"), # The exchange rate of the NTD to the USD
        col("Tags.JobId").alias("job_id")
    ).filter(
        (col("Tags.JobId").isNotNull())
        & (col("MeterCategory").isin("Virtual Machines", "Storage", "Bandwidth"))
    )

df = (
    df.groupBy("usage_date", "meter_category", "job_id").sum("cost")
    .withColumnRenamed("sum(cost)", "cost")
)

usage_df = (
    spark.table("system.billing.usage")
    .select("workspace_id", "usage_metadata.job_id")
    .filter(col("usage_metadata.job_id").isNotNull())
    .distinct()
)

df = (
    df.join(usage_df, df.job_id == usage_df.job_id, "inner")
    .select(df.usage_date, usage_df.workspace_id, df.job_id, df.meter_category, df.cost)
)

# display(df)

In [0]:
df.write.mode("overwrite").saveAsTable("main.billing.azure_cost")